# Fluid Surface Dynamics - Demo
## 유체 표면 역학 시뮬레이션

Based on: Galeano-Rios et al. (2017), J. Fluid Mech. 826, pp. 97-127

In [ ]:
# Import package
import sys
sys.path.append('..')  # Add parent directory to path

import fluid_dynamics as fd
import numpy as np
import matplotlib.pyplot as plt

# Show package info
fd.info()

## 1. Quick Start - 빠른 시작

Preset 설정으로 바로 실행

In [ ]:
# Quick run with preset
solver = fd.quick_run("fast", backend="auto", show_plots=True)

## 2. Custom Configuration - 커스텀 설정

파라미터를 직접 설정하여 실행

In [ ]:
# Create custom configuration
config = fd.SimulationConfig(
    physical=fd.PhysicalParameters(
        L_cm=50.0,      # Domain length (cm)
        D_cm=5.0,       # Depth (cm)
        nu=0.00894      # Viscosity (cm²/s)
    ),
    numerical=fd.NumericalParameters(
        nx=200,         # Grid points in x
        nz=40,          # Grid points in z
        nt=1000,        # Time steps
        total_time=5.0  # Total time (s)
    ),
    initial=fd.InitialCondition(
        type="gaussian",
        amplitude=0.05,
        width_factor=0.1
    ),
    compute=fd.ComputeConfig(
        backend="auto",  # "cpu", "gpu", or "auto"
        verbose=True
    )
)

# Show configuration summary
config.summary()

In [ ]:
# Create solver and run
solver = fd.FluidSolver(config, backend="auto")
final_state = solver.run()

print(f"\nFinal time: {final_state.time:.3f}s")
print(f"Final max|η|: {np.max(np.abs(final_state.eta)):.6f}")

## 3. Visualization - 시각화

다양한 진단 플롯

In [ ]:
# Comprehensive diagnostic plot
fig = fd.plot_diagnostics(solver, save_path="diagnostics.png")
plt.show()

In [ ]:
# Individual plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

fd.plot_surface_evolution(solver, ax=axes[0, 0])
fd.plot_velocity_field(solver, ax=axes[0, 1])
fd.plot_energy(solver, ax=axes[1, 0])
fd.plot_phase_space(solver, ax=axes[1, 1])

plt.tight_layout()
plt.show()

In [ ]:
# Create animation (takes some time)
anim = fd.create_animation(solver, fps=10, save_path="animation.gif")
print("Animation saved!")

## 4. Boundary Condition Verification - 경계조건 검증

Non-slip BC가 잘 적용되었는지 확인

In [ ]:
# Verify boundary conditions
bc_check = solver.verify_boundary_conditions()

print("Boundary Condition Check:")
print(f"  Bottom u_x: {bc_check['bottom_u_x']:.2e} (should be ~0)")
print(f"  Bottom u_z: {bc_check['bottom_u_z']:.2e} (should be ~0)")
print(f"  Passes: {bc_check['passes']}")

# Plot bottom velocity
u_x, u_z = solver.compute_velocity_field()
x_vec = np.linspace(0, solver.grid['L'], solver.nx, endpoint=False)

fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].plot(x_vec, u_x[0, :], linewidth=2)
axes[0].set_title('Bottom Horizontal Velocity $u_x$')
axes[0].set_xlabel('x')
axes[0].set_ylabel('$u_x$')
axes[0].grid(True, alpha=0.3)

axes[1].plot(x_vec, u_z[0, :], linewidth=2)
axes[1].set_title('Bottom Vertical Velocity $u_z$')
axes[1].set_xlabel('x')
axes[1].set_ylabel('$u_z$')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Parameter Sweep - 파라미터 스윕

점성(viscosity)을 변화시키며 시뮬레이션

In [ ]:
# Run parameter sweep for viscosity
base_config = fd.get_preset("fast")
viscosities = [0.00894, 0.02, 0.05, 0.1]  # cm²/s

results = fd.run_parameter_sweep(
    base_config, 
    "physical.nu", 
    viscosities,
    backend="auto"
)

In [ ]:
# Plot results
fig, axes = plt.subplots(1, 3, figsize=(18, 4))

fd.plot_parameter_sweep(results, "Viscosity (cm²/s)", 
                       metric="max_eta", ax=axes[0])
fd.plot_parameter_sweep(results, "Viscosity (cm²/s)", 
                       metric="final_eta_center", ax=axes[1])
fd.plot_parameter_sweep(results, "Viscosity (cm²/s)", 
                       metric="energy_decay", ax=axes[2])

plt.tight_layout()
plt.show()

## 6. Custom Initial Condition - 커스텀 초기조건

In [ ]:
# Define custom initial condition
def two_bumps(x):
    """Two Gaussian bumps"""
    L = x[-1] - x[0] if len(x) > 1 else 10
    eta = 0.05 * np.exp(-(x - L/3)**2 / (2*(L/15)**2))
    eta += 0.03 * np.exp(-(x - 2*L/3)**2 / (2*(L/15)**2))
    return eta

# Create config with custom IC
config_custom = fd.get_preset("fast")
config_custom.initial = fd.InitialCondition(
    type="custom",
    custom_function=two_bumps
)

# Run
solver_custom = fd.FluidSolver(config_custom)
solver_custom.run()

# Plot
fd.plot_surface_evolution(solver_custom)
plt.title("Custom IC: Two Bumps")
plt.show()

## 7. Performance Comparison - 성능 비교

CPU vs GPU (if available)

In [ ]:
import time

config_perf = fd.get_preset("default")
config_perf.compute.verbose = False

# CPU
print("Testing CPU...")
t0 = time.time()
solver_cpu = fd.FluidSolver(config_perf, backend="cpu")
solver_cpu.run(save_history=False)
t_cpu = time.time() - t0
print(f"CPU time: {t_cpu:.2f}s ({config_perf.numerical.nt/t_cpu:.1f} steps/sec)")

# GPU (if available)
try:
    print("\nTesting GPU...")
    t0 = time.time()
    solver_gpu = fd.FluidSolver(config_perf, backend="gpu")
    solver_gpu.run(save_history=False)
    t_gpu = time.time() - t0
    print(f"GPU time: {t_gpu:.2f}s ({config_perf.numerical.nt/t_gpu:.1f} steps/sec)")
    print(f"\nSpeedup: {t_cpu/t_gpu:.2f}x")
except Exception as e:
    print(f"GPU not available: {e}")

## 8. Save and Load Results - 결과 저장/불러오기

In [ ]:
import pickle

# Save solver
with open('solver_results.pkl', 'wb') as f:
    pickle.dump({
        'config': solver.config,
        'history': solver.history,
        'final_state': solver.state,
        'timings': solver.timings
    }, f)
print("Results saved to solver_results.pkl")

# Load solver
with open('solver_results.pkl', 'rb') as f:
    saved = pickle.load(f)
print(f"Loaded results: {len(saved['history']['eta'])} time points")

## Next Steps

- Modify parameters in Section 2
- Try different initial conditions in Section 6
- Run parameter sweeps for other variables
- Use `examples/run_simulation.py` for batch jobs
- Submit to SLURM with `examples/slurm_job.sh`